### Background
In this notebook we select the best compounds from the latest round of synthesis + the best compounds from the original series 4 to seed chemsampler and generate new molecules. Those will be scored against malaria predictive models as well as other relevant aspects (admet...)

In [1]:
import os
import pandas as pd

DATAPATH = "../data"

In [2]:
filename = "ersilia_nov22.csv"
df = pd.read_csv(os.path.join(DATAPATH, filename))
df = df[df["activity"]<1]
smi = df["smiles"].tolist()

In [3]:
from chemsampler import ChemSampler

cs = ChemSampler(samplers_list= ["ChemblSampler", "PubChemSampler", "StonedSampler", "FasmifraSampler", "SmallWorldSampler", "MollibSampler"])
results = cs.sample(smi, num_samples=10000)

ChemblSampler


 25%|██▌       | 1/4 [00:10<00:31, 10.46s/it]

ChEMBL seems to be down right now


 25%|██▌       | 1/4 [00:20<01:02, 20.67s/it]


ChEMBL seems to be down right now
0
ChemblSampler


...

Using TensorFlow backend.
Using TensorFlow backend.
Using TensorFlow backend.
Using TensorFlow backend.


Sampling from model saved at epoch 04
Sampling from model saved at epoch 08
Sampling from model saved at epoch 06
Sampling from model saved at epoch 02
Status for model from epoch 08: 10% of the molecules sampled in 5231.71 seconds
Status for model from epoch 06: 10% of the molecules sampled in 5364.18 seconds
Status for model from epoch 04: 10% of the molecules sampled in 5522.12 seconds
Status for model from epoch 02: 10% of the molecules sampled in 5795.95 seconds
Status for model from epoch 08: 20% of the molecules sampled in 4846.78 seconds
Status for model from epoch 06: 20% of the molecules sampled in 4918.74 seconds
Status for model from epoch 04: 20% of the molecules sampled in 4861.00 seconds
Status for model from epoch 02: 20% of the molecules sampled in 4788.62 seconds
Status for model from epoch 08: 30% of the molecules sampled in 4379.47 seconds
Status for model from epoch 02: 30% of the molecules sampled in 4703.94 seconds
Status for model from epoch 06: 30% of the molec

Using TensorFlow backend.


Sampling from model saved at epoch 10
Status for model from epoch 02: 100% of the molecules sampled in 4940.25 seconds
SAMPLING DONE for model from epoch 02 in 50432.28 seconds
Status for model from epoch 06: 100% of the molecules sampled in 5033.53 seconds
SAMPLING DONE for model from epoch 06 in 50672.97 seconds
Status for model from epoch 04: 100% of the molecules sampled in 4542.22 seconds
SAMPLING DONE for model from epoch 04 in 51609.56 seconds
Status for model from epoch 10: 10% of the molecules sampled in 3359.84 seconds
Status for model from epoch 10: 20% of the molecules sampled in 2476.87 seconds
Status for model from epoch 10: 30% of the molecules sampled in 2357.30 seconds
Status for model from epoch 10: 40% of the molecules sampled in 2416.46 seconds
Status for model from epoch 10: 50% of the molecules sampled in 2446.57 seconds
Status for model from epoch 10: 60% of the molecules sampled in 2336.97 seconds
Status for model from epoch 10: 70% of the molecules sampled in 2

4it [00:27,  6.98s/it]


In [4]:
for k,v in results.items():
    print(k)
    print(len(v))

ChemblSampler
32
PubChemSampler
12
StonedSampler
168
FasmifraSampler
7
SmallWorldSampler
2
MollibSampler
107


In [5]:
import csv
with open(os.path.join(DATAPATH, "sampled_{}.csv".format(filename)), 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["smiles"])
    for i in range(len(results)):
        key = list(results.keys())[i]
        # writer.writerow([key]) #it will write a single list, if want to retrieve by sampler, uncomment this
        for j in range(len(results[key])):
            writer.writerow([results[key][j]])

# New compounds analysis

In [2]:
#find the Maximum common substructure in the set of tested smiles
from rdkit import Chem
import rdkit.Chem.rdFMCS as rdFMCS

intsmi = pd.read_csv(os.path.join(DATAPATH, "ersilia_nov22.csv"))

mols = [Chem.MolFromSmiles(smi) for smi in intsmi["smiles"]]
mcs = rdFMCS.FindMCS(mols)
smarts1 = mcs.smartsString
#a bit more flexibility has been allowed in the right hand substituent
smarts2 = "F[#6](F)-[#8]-c1ccc(cc1)-c1nnc2cncc(-[#8]-[#6]-[*])n12"

In [3]:
#test the different smarts to identify best
smiles_list = ["OCN1N=NC(C2=CC=C(OCC3=CC=C(OCC4=CC=CC(C(F)(F)F)=C4)C=C3)C=C2)=N1",
    "FC(F)OC1=CC=C(C2=NN(CC3=CC=CC(C(F)(F)F)=C3)C=N2)C=C1",
    "C1=2C=NC=C(N1C(=NN=2)C3=CC=C(C=C3)OC(F)F)OCCC4C=5C=C(F)C(=CC=5C4)F",
    "O(C)C(C=1C=CC(=CC=1)C(F)F)COC=2N3C(C=NC=2)=NN=C3C=4C=CC=CC=4",
    "C=NC(C1=CC=C(C=C1)OC(F)F)OCOC2=CN=CC=3N2C(=NN=3)C=4C=CC(OC(F)F)=CC=4",
    "CC(F)OC1=CC=C(C=C1)C1=NN=C2C=NC=C(CCC3=CC=CC(=C3)C(F)(F)F)N12"]

pattern = Chem.MolFromSmarts(smarts2)
for idx,smiles in enumerate(smiles_list):
    m = Chem.MolFromSmiles(smiles)
    print("Structure {}: pattern found {}".format(idx+1,m.HasSubstructMatch(pattern)))

Structure 1: pattern found False
Structure 2: pattern found False
Structure 3: pattern found True
Structure 4: pattern found False
Structure 5: pattern found True
Structure 6: pattern found False


In [14]:
#for all sampled smiles
sampled_file = "sampled_ersilia_nov22.csv"

df = pd.read_csv(os.path.join(DATAPATH, sampled_file))
patterns = [smarts1, smarts2]

for i,p in enumerate(patterns):
    pmol = Chem.MolFromSmarts(p)
    substr = []
    for smi in df["smiles"]:
        m = Chem.MolFromSmiles(smi)
        if m.HasSubstructMatch(pmol):
            substr += [1]
        else:
            substr += [0]
    df["substr{}".format(i)] = substr

In [ ]:
# Predictions on the sampled molecules
preds_1 = pd.read_csv(os.path.join(DATAPATH, "zairachem_results", "osm_1_sampled_preds.csv"))
preds_25 = pd.read_csv(os.path.join(DATAPATH, "zairachem_results", "osm_25_sampled_preds.csv"))

df["act_1"] = preds_1["clf_ex1"]
df["act_2.5"] = preds_25["clf_ex1"]

In [ ]:
#Other filters
eos2r5a
eos7a45

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1, figsize=(5,5))
ax.hist(preds_1["clf_ex1"], histtype="step")
ax.hist(preds_25["clf_ex1"], histtype="step")

In [ ]:
#quick comparison with the range of values we get from the test split
test_1 = pd.read_csv(os.path.join(DATAPATH, "zairachem_results", "osm_1_test.csv"))
test_25 = pd.read_csv(os.path.join(DATAPATH, "zairachem_results", "osm_25_test.csv"))
fig,ax = plt.subplots(1,1, figsize=(5,5))
ax.hist(test_1["clf_ex1"], histtype="step")
ax.hist(test_25["clf_ex1"], histtype="step")